<a href="https://colab.research.google.com/github/RaffiSatamyan/GeoLifeCLEF-2024-LifeCLEF-CVPR-FGVC-Kaggle-Competition-/blob/main/Preprocess_%2B_visualize_spatial_data_%2B_eda_%2B_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'geolifeclef-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64733%2F7762802%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240329%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240329T160258Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5295510d15ece2eeb24c18e6495af1a9bd1762a13cb2b02fe9bfa193e56d87076b7aa5c33197ff150f981f05570d5f3dee3567024cd2afd2de24debe61b83e3a2b337de62691881976ac4479fd24e670a0ff09fc8085a5f6a052bfa280a050e060abb7443b18c5f8ec339baa23335a17936e28abe44820bb7e42ea0de279fd829c812a2666730cab5f1d9d3fcdf023ddf99de4b04b0933db6ecf597111261ab06e3aa0ff88b49da094100deee88e49f4d0c605a16132686390bb0804a33d753daa645f0707a97242294cbfc40cd853b1f1901498982b59d4ee00a61430eccc290d50280a13c8e17477b8faff50618cb3c6d87d543635840c0a14723ae28a7619,climateclef:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4533016%2F7752763%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240329%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240329T160259Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5c92cc3945000ac4aca660d900edbdd86b96932b10f5c5aa782509cb412e3eee99655e9aed7eb65b5c681c04228806db0f2dcb75e4bcd70741079969c27ddb2b5412ac6e725244964494034d1c6479c53e11fd84fafba9405c2d1f64de06d6223c97a25d4bbd038f9e21832ea3c68f4246f96e9d0f817e73fa6ab16bdea278ad069c1d05b349f57e2b6849f728d9bafe0fcf60774461f506e25cb84b2c533d12bd165983f4bf5fb12c8fba5f4798b2f450f3d69772ffb4b44ab0409aeeb707c1ec40e6d90909fb4888f651f70779035d97ac5c9f99686c4d91e3a7eebbbab32fe422328b02084dbdcf9f1a2f7ff0caf56d2205a667c5794ca693356b56e32966'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1493525327 bytes downloaded
Downloaded and uncompressed: geolifeclef-2024
[============                                      ] 6446080000 bytes downloaded

In [ ]:
from glob import glob
import os
# from netCDF4 import Dataset
import pandas as pd
import geopandas as gpd #
from shapely.geometry import Polygon, LineString, Point
import rasterio
from rasterio.plot import show
# from rasterio.transform import from_origin
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
import cv2
import numpy as np
import geopandas as gpd
import tqdm

root_path = '/kaggle/input'

### Read rasterfile and Visualize with point vector

In [ ]:
# Read rasterfile
bio_raster_path_list = glob(root_path + '/climateclef/Climate/BioClimatic_Average_1981-2010/*.tif')
climatic_raster_path_list = glob(root_path + '/climateclef/Climate/Climatic_Monthly_2000-2019/*.tif')

In [ ]:
# plot raster file
for i,raster_path in enumerate(climatic_raster_path_list[:2]):
    with rasterio.open(raster_path) as src:
        plt.figure(figsize = [8,8])
        plt.title(raster_path.split("/")[-1])
        show(src)

In [ ]:
# plot raster file
for i,raster_path in enumerate(bio_raster_path_list[:2]):
    with rasterio.open(raster_path) as src:
        plt.figure(figsize = [8,8])
        plt.title(raster_path.split("/")[-1])
        show(src)

In [ ]:
# visualize every monitoring site in meta data
# Example for 1000 samples
# Read meta data
train_meta = pd.read_csv(root_path + "/geolifeclef-2024/GLC24_PA_metadata_train.csv")
train_meta.head()

sub_train_meta = train_meta.drop_duplicates('surveyId').sample(n=1000, random_state=42)
sub_train_meta.index = range(len(sub_train_meta))

# make Point vector
point_list = []
for i in tqdm.tqdm(range(len(sub_train_meta))):
    x,y = sub_train_meta.loc[i, ['lon', 'lat']]
    poind_i = Point(x,y)
    point_list.append(poind_i)

sub_train_meta.loc[:,'geometry'] = point_list

# Read meta data
test_meta = pd.read_csv(root_path + "/geolifeclef-2024/GLC24_PA_metadata_test.csv")
test_meta.head()

sub_test_meta = test_meta.drop_duplicates('surveyId').sample(n=1000, random_state=42)
sub_test_meta.index = range(len(sub_test_meta))

# make Point vector
point_list = []
for i in tqdm.tqdm(range(len(sub_test_meta))):
    x,y = sub_test_meta.loc[i, ['lon', 'lat']]
    poind_i = Point(x,y)
    point_list.append(poind_i)

sub_test_meta.loc[:,'geometry'] = point_list

In [ ]:
train_sub_meta_gdf = gpd.GeoDataFrame(sub_train_meta, geometry = 'geometry')
train_sub_meta_gdf.crs = {'init':'epsg:4326'}
train_sub_meta_gdf.head()

test_sub_meta_gdf = gpd.GeoDataFrame(sub_test_meta, geometry = 'geometry')
test_sub_meta_gdf.crs = {'init':'epsg:4326'}
test_sub_meta_gdf.head()

# visualize each monitoring site
m = train_sub_meta_gdf.drop_duplicates(['lon', 'lat']).explore(color = 'green')
test_sub_meta_gdf.drop_duplicates(['lon', 'lat']).explore(m=m, color = 'red')

In [ ]:
# visualize with raster file
fig, ax = plt.subplots(figsize=(8, 8))

raster_path = bio_raster_path_list[0]
with rasterio.open(raster_path) as src:
    show(src, ax=ax)
train_sub_meta_gdf.drop_duplicates(['lon', 'lat']).plot(ax=ax,color = 'green')
test_sub_meta_gdf.drop_duplicates(['lon', 'lat']).plot(ax=ax,color = 'red')

In [ ]:
test_meta

### merge raster data with meta data

In [ ]:
%%time
# two raster file example
train_geo_info_gdf = train_sub_meta_gdf.drop_duplicates(['lon', 'lat'])
train_geo_info_gdf.index = range(len(train_geo_info_gdf))

test_geo_info_gdf = test_sub_meta_gdf.drop_duplicates(['lon', 'lat'])
test_geo_info_gdf.index = range(len(test_geo_info_gdf))

for raster_path in tqdm.tqdm(bio_raster_path_list[:2]):
    bio_raster = rasterio.open(raster_path)

    bio_values = zonal_stats(vectors=train_geo_info_gdf.geometry,raster=bio_raster.read(1), affine=bio_raster.transform,stats=['mean'])
    bio_values_list = [value['mean'] for value in bio_values]
    feature_name = raster_path.split("/")[-1].split(".")[0]
    train_geo_info_gdf.loc[:,feature_name] = bio_values_list

    bio_values = zonal_stats(vectors=test_geo_info_gdf.geometry,raster=bio_raster.read(1), affine=bio_raster.transform,stats=['mean'])
    bio_values_list = [value['mean'] for value in bio_values]
    feature_name = raster_path.split("/")[-1].split(".")[0]
    test_geo_info_gdf.loc[:,feature_name] = bio_values_list

In [ ]:
train_geo_info_gdf.head()

In [ ]:
test_geo_info_gdf.head()

### Dataset merge
Utilize the CSV file provided in this competition


* landsat timeseries data has missing value
* If Landsat timeseries data is collected after the sampling date, it is considered as data not available for the originally intended prediction timeframe. If not treated as missing values, it may lead to data leakage issues.
* We need to consider this timeframe issue when merging datasets.

* make train dataset

In [ ]:
train_meta = pd.read_csv(root_path + "/geolifeclef-2024/GLC24_PA_metadata_train.csv")

In [ ]:
%%time
# read average landsat dataset
# Simply merge; this set has no missing values.
train_bioclimatic = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Climate/Average 1981-2010/GLC24-PA-train-bioclimatic.csv")
train_bioclimatic_monthly = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Climate/Monthly/GLC24-PA-train-bioclimatic_monthly.csv")
train_elevation = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Elevation/GLC24-PA-train-elevation.csv")
train_human_footprint = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Human Footprint/GLC24-PA-train-human_footprint.csv")
train_land_cover = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/LandCover/GLC24-PA-train-landcover.csv")
train_soilgrid = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/SoilGrids/GLC24-PA-train-soilgrids.csv")

# first merge bioclimatic averate dataset
merge_key = 'surveyId'

# merge tables
train_merged = pd.merge(train_bioclimatic, train_bioclimatic_monthly, on=merge_key)
train_merged = pd.merge(train_merged, train_elevation, on=merge_key)
train_merged = pd.merge(train_merged, train_human_footprint, on=merge_key)
train_merged = pd.merge(train_merged, train_land_cover, on=merge_key)
train_merged = pd.merge(train_merged, train_soilgrid, on=merge_key)

train_merged

In [ ]:
# landsat time series missing value check
landsat_example = pd.read_csv(root_path + f"/geolifeclef-2024/PA-train-landsat_time_series/GLC24-PA-train-landsat_time_series-blue.csv")
landsat_example.head() ## missing value!

In [ ]:
# We merge climatic timeseries data within a 10-year time window.(4 q * 10 years = 40)
# For example, if the survey year was 2018, we merge monthly data from the years 2008 to 2017.

def shift_and_slice(array):
    for i in range(len(array)):
        row = array[i]
        nan_indices = np.isnan(row)

        rightmost_nan_index = np.argmax(nan_indices)

        if np.any(nan_indices):
            shifted_row = np.roll(row, len(row) - rightmost_nan_index)
            array[i] = shifted_row

    return array[:,-(4 * 10):]

for monthly_landsat_data in tqdm.tqdm(['blue', 'green', 'red', 'nir', 'swir1', 'swir2']):
    landsat_timeseries = pd.read_csv(root_path + f"/geolifeclef-2024/PA-train-landsat_time_series/GLC24-PA-train-landsat_time_series-{monthly_landsat_data}.csv")
    origin_arr = landsat_timeseries.iloc[:,1:].values
    new_arr = shift_and_slice(origin_arr)

    window_dataset =  landsat_timeseries.iloc[:,0:1]
    window_dataset.loc[:,[f'{monthly_landsat_data}_t-{i+1}' for i in range(40)]] = new_arr

    train_merged = pd.merge(train_merged, window_dataset, on=merge_key)

In [ ]:
train_merged

In [ ]:
%%time
# species id range from 1 to 11254
print(np.max(train_meta.drop_duplicates('speciesId').loc[:,['speciesId']].values), np.min(train_meta.drop_duplicates('speciesId').loc[:,['speciesId']].values))
species_target_columns = [str(sp_id) for sp_id in range(1,11255)]
input_columns = train_merged.columns.tolist()[1:]

# reshape metadata
surveyId_list = train_meta.drop_duplicates('surveyId').loc[:,['surveyId']].values
target_arr = np.zeros([len(surveyId_list),11254])
for i,survey_id in tqdm.tqdm(enumerate(surveyId_list)):
    presense_species_ids = train_meta.loc[train_meta.surveyId == survey_id[0],'speciesId'].values
    target_arr[i,[int(ids)-1 for ids in presense_species_ids]] = 1

train_target_df = pd.DataFrame(target_arr)
train_target_df.columns = species_target_columns
train_target_df.loc[:,'surveyId'] = surveyId_list
train_target_df

* spatial interpolation for missing values

In [ ]:
# check missing values
missing_value_col_list = []
rows_with_missing_values = {}
for column in train_merged.columns:
    missing_rows = train_merged.index[train_merged[column].isnull()].tolist()
    if len(missing_rows) > 0:
        rows_with_missing_values[column] = len(missing_rows)
        missing_value_col_list.append(column)

print(rows_with_missing_values)

In [ ]:
# check missing value region
target_colname = 'swir2_t-39'
sub_df = pd.merge(train_merged.loc[:, ['surveyId', target_colname]], train_meta.loc[:, ['lon', 'lat', 'surveyId']].drop_duplicates("surveyId"), on='surveyId')
na_ind = sub_df.loc[:,target_colname].isna().values

sub_df_train = sub_df.loc[~na_ind,:]
sub_df_target = sub_df.loc[na_ind,:]
x,y,z = sub_df_train['lon'].values, sub_df_train['lat'].values, sub_df_train[target_colname].values
x_target,y_target,z_target = sub_df_target['lon'].values, sub_df_target['lat'].values, np.zeros(sub_df_target[target_colname].shape)

plt.figure(figsize=(12, 6))
plt.scatter(x, y, marker='o', color='green', label='Non Missing Data')
plt.scatter(x_target, y_target, marker='o', color='red', label='Missing Data', s = 5)

plt.legend()
plt.show()

In [ ]:
# check missing value region
target_colname = 'Elevation'
sub_df = pd.merge(train_merged.loc[:, ['surveyId', target_colname]], train_meta.loc[:, ['lon', 'lat', 'surveyId']].drop_duplicates("surveyId"), on='surveyId')
na_ind = sub_df.loc[:,target_colname].isna().values

sub_df_train = sub_df.loc[~na_ind,:]
sub_df_target = sub_df.loc[na_ind,:]
x,y,z = sub_df_train['lon'].values, sub_df_train['lat'].values, sub_df_train[target_colname].values
x_target,y_target,z_target = sub_df_target['lon'].values, sub_df_target['lat'].values, np.zeros(sub_df_target[target_colname].shape)

plt.figure(figsize=(12, 6))
plt.scatter(x, y, marker='o', color='green', label='Non Missing Data')
plt.scatter(x_target, y_target, marker='o', color='red', label='Missing Data')

plt.legend()
plt.show()

In [ ]:
# check missing value region
target_colname = 'HumanFootprint-Roads'
sub_df = pd.merge(train_merged.loc[:, ['surveyId', target_colname]], train_meta.loc[:, ['lon', 'lat', 'surveyId']].drop_duplicates("surveyId"), on='surveyId')
na_ind = sub_df.loc[:,target_colname].isna().values

sub_df_train = sub_df.loc[~na_ind,:]
sub_df_target = sub_df.loc[na_ind,:]
x,y,z = sub_df_train['lon'].values, sub_df_train['lat'].values, sub_df_train[target_colname].values
x_target,y_target,z_target = sub_df_target['lon'].values, sub_df_target['lat'].values, np.zeros(sub_df_target[target_colname].shape)

plt.figure(figsize=(12, 6))
plt.scatter(x, y, marker='o', color='green', label='Non Missing Data')
plt.scatter(x_target, y_target, marker='o', color='red', label='Missing Data', s = 5)

plt.legend()
plt.show()

In [ ]:
# check missing value region
target_colname = 'Soilgrid-bdod'
sub_df = pd.merge(train_merged.loc[:, ['surveyId', target_colname]], train_meta.loc[:, ['lon', 'lat', 'surveyId']].drop_duplicates("surveyId"), on='surveyId')
na_ind = sub_df.loc[:,target_colname].isna().values

sub_df_train = sub_df.loc[~na_ind,:]
sub_df_target = sub_df.loc[na_ind,:]
x,y,z = sub_df_train['lon'].values, sub_df_train['lat'].values, sub_df_train[target_colname].values
x_target,y_target,z_target = sub_df_target['lon'].values, sub_df_target['lat'].values, np.zeros(sub_df_target[target_colname].shape)

plt.figure(figsize=(12, 6))
plt.scatter(x, y, marker='o', color='green', label='Non Missing Data')
plt.scatter(x_target, y_target, marker='o', color='red', label='Missing Data', s = 5)

plt.legend()
plt.show()

* Based on observations from the plot, it appears that there are densely populated measurements near the missing regions.
* To handle interpolation, a straightforward approach involves using the values from the nearest neighbors. If a more reliable method is desired, considering techniques such as kriging is also worth exploring.

In [ ]:
%%time
# missing_value_col_list
for target_colname in tqdm.tqdm(missing_value_col_list):

    sub_df = pd.merge(train_merged.loc[:, ['surveyId', target_colname]], train_meta.loc[:, ['lon', 'lat', 'surveyId']].drop_duplicates("surveyId"), on='surveyId')
    na_ind_bool = sub_df.loc[:,target_colname].isna().values

    na_ind = sub_df.index[na_ind_bool]
    no_na_ind = sub_df.index[~na_ind_bool]



    given_points = sub_df.iloc[no_na_ind,[2,3]].values

    for na_ind_i in na_ind:

        target_point = sub_df.iloc[na_ind_i,[2,3]].values
        distances = np.linalg.norm(given_points - target_point, axis=1)
        closest_index = no_na_ind[np.argmin(distances)]
        closest_value = sub_df.iloc[closest_index,1]
        train_merged.loc[na_ind_i, target_colname] = closest_value


In [ ]:
# check missing values
rows_with_missing_values = {}
for column in train_merged.columns:
    missing_rows = train_merged.index[train_merged[column].isnull()].tolist()
    if len(missing_rows) > 0:
        rows_with_missing_values[column] = len(missing_rows)
print(rows_with_missing_values)

In [ ]:
# check -inf values
columns_with_negative_inf = train_merged.columns[train_merged.apply(lambda col: any(col == float('-inf')))]
if len(columns_with_negative_inf) > 0:
    print("This column contains a value of -inf")
    for column_name in columns_with_negative_inf:
        print(column_name)
else:
    print("no inf value")


In [ ]:
# Replace -inf with zero.
train_merged.replace({float('-inf'): 0}, inplace=True)

# check -inf values
columns_with_negative_inf = train_merged.columns[train_merged.apply(lambda col: any(col == float('-inf')))]
if len(columns_with_negative_inf) > 0:
    print("This column contains a value of -inf")
    for column_name in columns_with_negative_inf:
        print(column_name)
else:
    print("no inf value")

* make test dataset

In [ ]:
%%time
# read average landsat dataset
# Simply merge; this set has no missing values.
test_bioclimatic = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Climate/Average 1981-2010/GLC24-PA-test-bioclimatic.csv")
test_bioclimatic_monthly = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Climate/Monthly/GLC24-PA-test-bioclimatic_monthly.csv")
test_elevation = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Elevation/GLC24-PA-test-elevation.csv")
test_human_footprint = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Human Footprint/GLC24-PA-test-human_footprint.csv")
test_land_cover = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/LandCover/GLC24-PA-test-landcover.csv")
test_soilgrid = pd.read_csv(root_path + "/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/SoilGrids/GLC24-PA-test-soilgrids.csv")

# first merge bioclimatic averate dataset
merge_key = 'surveyId'

# merge tables
test_merged = pd.merge(test_bioclimatic, test_bioclimatic_monthly, on=merge_key)
test_merged = pd.merge(test_merged, test_elevation, on=merge_key)
test_merged = pd.merge(test_merged, test_human_footprint, on=merge_key)
test_merged = pd.merge(test_merged, test_land_cover, on=merge_key)
test_merged = pd.merge(test_merged, test_soilgrid, on=merge_key)

test_merged

In [ ]:
# We merge climatic timeseries data within a 10-year time window.(4 q * 10 years = 40)
# For example, if the survey year was 2018, we merge monthly data from the years 2008 to 2017.
for monthly_landsat_data in tqdm.tqdm(['blue', 'green', 'red', 'nir', 'swir1', 'swir2']):
    landsat_timeseries = pd.read_csv(root_path + f"/geolifeclef-2024/PA-test-landsat_time_series/GLC24-PA-test-landsat_time_series-{monthly_landsat_data}.csv")
    origin_arr = landsat_timeseries.iloc[:,1:].values
    new_arr = shift_and_slice(origin_arr)

    window_dataset =  landsat_timeseries.iloc[:,0:1]
    window_dataset.loc[:,[f'{monthly_landsat_data}_t-{i+1}' for i in range(40)]] = new_arr

    test_merged = pd.merge(test_merged, window_dataset, on=merge_key)

In [ ]:
test_merged

* train and prediction example for speciesId 540

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import xgboost as xgb

In [ ]:
# train
tartget_speciesId = '540'

train_df_sub = pd.merge(train_target_df.loc[:,['surveyId', tartget_speciesId]], train_merged, on='surveyId')
train_df_sub.head()

X, y = train_df_sub.iloc[:,2:].values, train_df_sub.iloc[:,1].values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=7)

In [ ]:
plt.hist(y)

In [ ]:
%%time
model = XGBClassifier(objective='binary:logistic',eval_metric=['error','logloss'], verbose = 1)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=True,)

In [ ]:
test_merged

* prediction and mapping species distributions

In [ ]:
test_meta = pd.read_csv(root_path + "/geolifeclef-2024/GLC24_PA_metadata_test.csv")
X_test = test_merged.iloc[:,1:].values
y_pred_proba = model.predict_proba(X_test)[:,1]
test_meta.loc[:,tartget_speciesId + "_prob"] = y_pred_proba

In [ ]:
test_meta

In [ ]:
# make Point vector
point_list = []
for i in tqdm.tqdm(range(len(test_meta))):
    x,y = test_meta.loc[i, ['lon', 'lat']]
    poind_i = Point(x,y)
    point_list.append(poind_i)

test_meta.loc[:,'geometry'] = point_list

In [ ]:
test_meta_gdf = gpd.GeoDataFrame(test_meta, geometry = 'geometry')
test_meta_gdf.crs = {'init':'epsg:4326'}

In [ ]:
test_meta_gdf.explore(column = tartget_speciesId + "_prob", cmap = 'jet', legend = True)

* feature importance

In [ ]:
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer(X_test)
features_col_list = train_merged.columns[1:]

shap.summary_plot(shap_values, X_test, feature_names = features_col_list)

* feature importance map Bio-pr_04_2013

In [ ]:
target_feature = 'Bio-pr_04_2013'
feature_shap_values = shap_values[:,features_col_list == target_feature].values

In [ ]:
feature_shap_values.shape

In [ ]:
test_meta_gdf.loc[:,'feature_contribution'] = feature_shap_values

In [ ]:
test_meta_gdf.explore(column = 'feature_contribution', cmap = 'jet', legend = True)

* feature importance map elevation

In [ ]:
Ftarget_feature = 'Elevation'
feature_shap_values = shap_values[:,features_col_list == target_feature].values
test_meta_gdf.loc[:,'feature_contribution'] = feature_shap_values
test_meta_gdf.explore(column = 'feature_contribution', cmap = 'jet', legend = True)